In [4]:
import sys
sys.path.append("..")
sys.path.append("c://users//mccar//miniconda3//lib//site-packages")
from config import set_environment
set_environment()

In [1]:
# !pip install -U accelerate bitsandbytes datasets transformers peft trl sentencepiece wandb langchain

In [2]:
# !pip show accelerate bitsandbytes datasets transformers peft trl sentencepiece wandb langchain

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import os
os.environ["WANDB_PROJECT"] = "finetuning"

In [8]:
import os
os.environ["WANDB_PROJECT"] = "finetuning"
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


ServiceStartProcessError: The wandb service process exited with 1. Ensure that `sys.executable` is a valid python interpreter. You can override it with the `_executable` setting or with the `WANDB__EXECUTABLE` environment variable.

In [17]:
if wandb.run is not None:
  wandb.finish()

In [9]:
from datasets import load_dataset

dataset_name = "squad_v2"
dataset = load_dataset(dataset_name, split="train")
eval_dataset = load_dataset(dataset_name, split="validation")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [10]:
load_dataset(dataset_name)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [11]:
dataset.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [14]:
model_id = "openlm-research/open_llama_3b_v2"
# new_model_name = f"openllama-3b-peft-{dataset_name}"

In [16]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
device_map="auto"

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
base_model.config.use_cache = False

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [18]:
output_dir = "/content/gdrive/My Drive/results"

In [20]:
from transformers import AutoTokenizer, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig
from trl import SFTTrainer

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# if you don't want to use google drive, just set this to a directory on your computer:
output_dir = "/content/gdrive/My Drive/results"
new_model_name = f"openllama-3b-peft-{dataset_name}"

training_args = TrainingArguments(
    output_dir=output_dir,  # use new_model_name if saving to hf
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=2000,  # training can still improve after many steps!
    num_train_epochs=100,
    evaluation_strategy="steps",
    eval_steps=5,  # update steps between two evaluations
    save_total_limit=5,  # only last 5 models are saved
    push_to_hub=False,  # you can set this to true if you want to upload your model to huggingspace
    load_best_model_at_end=True,  # to use in combination with early stopping
    report_to="wandb"
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="question",  # this depends on the dataset!
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=200)]
)

trainer.train()

ImportError: cannot import name 'file_exists' from 'huggingface_hub' (C:\Users\mccar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\__init__.py)

In [21]:
trainer.model.save_pretrained(
    os.path.join(output_dir, "final_checkpoint"),
)

NameError: name 'trainer' is not defined

In [22]:
trainer.model.push_to_hub(
    repo_id=new_model_name
)

NameError: name 'trainer' is not defined

In [23]:
# usually, the peft model is stored as an adapter, not as a full model, therefore the loading is a bit different:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

model_id = 'openlm-research/open_llama_3b_v2'
config = PeftConfig.from_pretrained("benji1a/openllama-3b-peft-squad_v2")
model = AutoModelForCausalLM.from_pretrained(model_id)
model = PeftModel.from_pretrained(model, "benji1a/openllama-3b-peft-squad_v2")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256
)
llm = HuggingFacePipeline(pipeline=pipe)

ImportError: cannot import name 'file_exists' from 'huggingface_hub' (C:\Users\mccar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\__init__.py)

In [ ]:
# Fine-tuning a PaLM model for text classification can be done like this:
from skllm.models.palm import PaLMClassifier
clf = PaLMClassifier(n_update_steps=100)
clf.fit(X_train, y_train) # y_train is a list of labels
labels = clf.predict(X_test)

In [ ]:
# Similarly, you can fine-tune the GPT-3.5 model for text classification like this:
from skllm.models.gpt import GPTClassifier
clf = GPTClassifier(
base_model = "gpt-3.5-turbo-0613",
n_epochs = None, # int or None. When None, will be determined automatically by OpenAI
default_label = "Random", # optional
)
clf.fit(X_train, y_train) # y_train is a list of labels
labels = clf.predict(X_test)